# 前言
通过本文，你可以：
 * 了解AppBuilder KnowledgeBase MCP Server是什么
 * 了解如何通过AppBuilder-SDK调用AppBuilder KnowledgeBase MCP Server
 * 了解如何通过Cursor调用AppBuilder KnowledgeBase MCP Server

如果你需要了解更多关于AppBuilder结合MCP生态的功能,可跳转到下面的文档：
- 主动调用：通过AppBuilder-Agent调用MCP Server
  - 代码态调用：[AppBuilder结合端云组件，调用本地MCP Server](./client.ipynb) 
  - 代码态调用：[AppBuilder结合地图MCP Server，实现智能导航](./baidu_map.ipynb)
  - 零代码态调用：[AppBuilder控制台使用MCP](https://cloud.baidu.com/doc/AppBuilder/s/Nm9vmpb3g)
- 被动调用：AppBuilder能力，融入MCP生态，可作为MCP Server提供能力
  - 组件能力：[AppBuilder组件转MCP Server](./server.ipynb)
  - 智能体能力：[AppBuilder Agent MCP Server](./app_mcp_server.ipynb)
  - AI搜索能力: [AppBuilder AI搜索MCP Server](https://github.com/baidubce/app-builder/tree/master/python/mcp_server/ai_search)
  - RAG能力：[AppBuilder知识库MCP Server](./knowledge_base_mcp_server.ipynb)


## AppBuilder Agent MCP Server简介
千帆AppBuilder提供了关于AppBuilder知识库的MCP Server，用户可通过MCP协议调用在AppBuilder中的知识库。当前包括以下能力：
- 创建知识库
- 获取知识库详情
- 获取知识库列表
- 知识库检索
- 上传知识库文档
- 获取文档列表

对于入门用户，还提供了环境配置、 api key获取等操作步骤。已熟悉用户可以直接跳过环境配置、api_key获取部分。

## 安装配置步骤

### 配置基础环境

#### 1、安装Python

大部分的MCP Server，例如Unity和Blender，都是Python SDK
安装Python最简单的方法是安装Miniconda
**注意：请确保安装的python>=3.12**

[Miniconda官网](https://www.anaconda.com/docs/getting-started/miniconda/install)

#### 2、下载Appbuilder-SDK

为了实现Appbuilder-SDK对AppBuilder Agent MCP Server的代码态调用，需要在conda环境下载appbuilder-sdk包


In [ ]:
!python3 -m pip install httpx appbuilder-sdk mcp


## 获取千帆AppBuilder API key

### 1、进入百度智能云千帆AppBuilder主页

进入百度智能云千帆AppBuilder主页，免费试用

- [百度智能云千帆AppBuilder主页](https://qianfan.cloud.baidu.com/appbuilder)


### 7、创建Appbuilder API key

- [Appbuilder API key授权](https://cloud.baidu.com/doc/AppBuilder/s/lm68r8e6i)

- [Appbuilder API 控制台管理中心](https://console.bce.baidu.com/iam/#/iam/apikey/list)

在【API Key】界面中，创建一个新的Key，并添加知识库的读写权限，记下来。

![](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E5%88%9B%E5%BB%BAAPPBUILDER-TOKEN.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2025-04-17T14%3A49%3A27Z%2F-1%2Fhost%2F9266ac8c93a262d2905811877d680f53af26668a0ee09e1b3a61c0b721de253c)

![api_key_knowledgebase](./image/api_key_knowledgebase.png)

## 使用AppBuilder SDK调用

In [ ]:
import asyncio
import json
import io
import os
import appbuilder
from appbuilder.mcp_server.client import MCPClient

os.environ["APPBUILDER_TOKEN"] = "YOUR_APPBUILDER_TOKEN"

async def main():
    mcp_client = MCPClient()
    await mcp_client.connect_to_server(service_url=service_url)

    # 创建知识库
    result = await mcp_client.call_tool(
        "create_knowledge_base",
        {"name": "mcp测试可删", "description": "mcp测试，可删"},
    )
    knowledge_base_info = json.loads(result.content[0].text)
    knowledge_base_id = knowledge_base_info.get("id")
    assert knowledge_base_id is not None
    appbuilder.logger.debug(f"create knowledge base success: {knowledge_base_id}")

    # 获取知识库详情
    result = await mcp_client.call_tool(
        "describe_knowledge_base",
        {"id": knowledge_base_id},
    )
    knowledge_base_info = json.loads(result.content[0].text)
    knowledge_base_id = knowledge_base_info.get("id")
    assert knowledge_base_id is not None
    appbuilder.logger.debug(f"describe knowledge base success: {knowledge_base_id}")

    # 获取知识库列表
    result = await mcp_client.call_tool(
        "list_knowledge_bases",
        {"max_keys": 10},
    )
    assert len(result.content) == 10
    appbuilder.logger.debug(f"list knowledge bases success: {len(result.content)}")

    # 上传文档到知识库
    file_content = "这里是你的文件内容字符串"
    with io.BytesIO(file_content.encode("utf-8")) as f:
        result = await mcp_client.call_tool(
            "upload_document",
            {"id": knowledge_base_id, "file_data": f, "file_name": "test.txt"},
        )
    document_info = json.loads(result.content[0].text)

    document_id = document_info.get("documentId")
    assert document_id is not None
    appbuilder.logger.debug(f"upload document success: {document_id}")

    # 获取文档列表
    result = await mcp_client.call_tool(
        "list_documents",
        {"id": knowledge_base_id},
    )
    appbuilder.logger.debug(f"list documents base success: {knowledge_base_id}")

    # 检索知识库
    result = await mcp_client.call_tool(
        "query_knowledge_base",
        {"query": "分子", "id_list": [knowledge_base_id]},
    )
    assert result.content[0].text is not None
    appbuilder.logger.debug("query knowledge base success")


if __name__ == "__main__":
    appbuilder.logger.setLoglevel("DEBUG")
    service_url = (
        "http://appbuilder.baidu.com/v2/ai_search/mcp/sse?api_key="
        + os.environ.get("APPBUILDER_TOKEN")
    )

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

用户选择其中需要的功能进行执行，比如创建知识库，打印如下：

```
DEBUG:appbuilder:create knowledge base success: 909*****-***a-****-****-*********2c93
```

## 使用Cursor调用
### Cursor配置
api_key=上面步骤获取到的api key，格式为"bce-v3/ALTAK..."

In [ ]:
{
    "mcpServers": {
        "AppBuilder RAG": {
            "url": "http://appbuilder.baidu.com/v2/knowledgeBase/mcp/sse?api_key=bce-v3/ALTAK..."
        }
    }
}

配置后，在Cursor设置页面可看到`create_knowledge_base`、`describe_knowledge_base`、`list_knowledge_bases`、`query_knowledge_base`、`upload_document`、`list_documents` 6个Tool。
![cursor config](./image/cursor_knowledgebase_config.png)

### 调用
![cursor knowledgebase](./image/cursor_knowledgebase_test.png)